In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from matplotlib.patches import Rectangle
import warnings
warnings.filterwarnings('ignore')

Считываем полученные данные - в обоих случаях генерировалось $10^8$ частиц

In [ ]:
base_dir = os.getcwd()

kev250 = pd.read_csv('output_files/250_kev_output_big.txt', sep=" ", header=None)
kev250.columns = ['trackNum', 'x', 'y', 'z', 'kinEn', 'IsFirstStepInVolume', 'particleName', 'volumeName', 'dose']
kev250.info()

In [ ]:
kev300 = pd.read_csv('output_files/300kev_output_big.txt', sep=" ", header=None)
kev300.columns = ['trackNum', 'x', 'y', 'z', 'kinEn', 'IsFirstStepInVolume', 'particleName', 'volumeName', 'dose']
kev300.info()

 Переносим систему отсчёта в центр мишени, оставляем только Гамма-кванты которые сделали первый шаг в объёме детектора

In [ ]:
kev250['z'] = kev250['z'] - 80
kev250 = kev250[(kev250['IsFirstStepInVolume'] == 1) & (kev250['particleName'] == 'gamma')]
kev250.count()

In [ ]:
kev300['z'] = kev300['z'] - 80
kev300 = kev300[(kev300['IsFirstStepInVolume'] == 1) & (kev300['particleName'] == 'gamma')]
kev300.count()

# Разделение таблиц по мишеням

In [ ]:
kev250_0 = kev250[kev250.volumeName == 'ScoringVolume']
kev250_1 = kev250[kev250.volumeName == 'ScoringPan07']
kev250_2 = kev250[kev250.volumeName == 'ScoringPan10']

kev300_0 = kev300[kev300.volumeName == 'ScoringVolume']
kev300_1 = kev300[kev300.volumeName == 'ScoringPan07']
kev300_2 = kev300[kev300.volumeName == 'ScoringPan10']

In [ ]:
lsm = np.polyfit(x=kev300_0.dose.index, y=kev300_0.dose.values, deg=1)

plt.figure(figsize=(16, 9))
sns.lineplot(kev250_0.dose, label = '250 кэВ')
sns.lineplot(kev300_0.dose, label = '300 кэВ')
sns.lineplot(x=kev300_0.dose.index, y = (lsm[0] * kev300_0.dose.index + lsm[1]), label = f'МНК: k = {lsm[0]:.1e}, b = {lsm[1]:.1e}')
plt.title('Доза в Греях - детектор на выходе из мишени')
plt.xlabel('Количество гамма-квантов вышедших из мишени')
plt.tight_layout()
plt.ylabel('Доза облучения (Грей)')
plt.grid()
plt.show()

kev250_0.dose.index.max()
# 1.6 * 10^(-10) - время, 1043068 - кол-во квантов на выходе - 515.014 грей за секунду - 300 кэВ
# 1.6 * 10^(-10) - время, 509633 - кол-во квантов на выходе - 251.631 грей за секунду - 250 кэВ

In [ ]:
lsm = np.polyfit(x=kev300_1.dose.index, y=kev300_1.dose.values, deg=1)

plt.figure(figsize=(16, 9))
sns.lineplot(kev250_1.dose, label = '250 кэВ')
sns.lineplot(kev300_1.dose, label = '300 кэВ')
sns.lineplot(x=kev300_1.dose.index, y = (lsm[0] * kev300_1.dose.index + lsm[1]), label = f'МНК: k = {lsm[0]:.1e}, b = {lsm[1]:.1e}')
plt.title('Доза в Греях - детектор на расстоянии 0.7 метр от мишени')
plt.xlabel('Количество гамма-квантов вышедших из мишени')
plt.ylabel('Доза облучения (Грей)')
plt.tight_layout()
plt.grid()
plt.show()

# 1.6 * 10^(-10) - время, 1043068 - кол-во квантов на выходе - 0.0372 грей за секунду - 300 кэВ
# 1.6 * 10^(-10) - время, 509633 - кол-во квантов на выходе - 0.0182 грей за секунду - 250 кэВ

In [ ]:
lsm = np.polyfit(x=kev300_2.dose.index, y=kev300_2.dose.values, deg=1)


plt.figure(figsize=(16, 9))
sns.lineplot(kev250_2.dose, label = '250 кэВ')
sns.lineplot(kev300_2.dose, label = '300 кэВ')
sns.lineplot(x=kev300_2.dose.index, 
            y = (lsm[0] * kev300_2.dose.index + lsm[1]), 
            label = f'МНК: k = {lsm[0]:.1e}, b = {lsm[1]:.1e}')

plt.title('Доза в Греях - детектор на расстоянии 1.0 метр от мишени')
plt.xlabel('Количество гамма-квантов вышедших из мишени')
plt.ylabel('Доза облучения (Грей)')
plt.tight_layout()
plt.grid()
plt.show()

# 1.6 * 10^(-10) - время, 1043068 - кол-во квантов на выходе - 0.0274 грей за секунду - 300 кэВ
# 1.6 * 10^(-10) - время, 509633 - кол-во квантов на выходе - 0.0134 грей за секунду - 250 кэВ

## Поверность координат детектирования частиц

Генерируется цилиндрический слой частиц с заданной энергией падающий на мишень по нормали

250 кэВ

In [ ]:
tmp = kev250[kev250['volumeName'] == 'ScoringVolume']
tmp = tmp.reset_index(drop=True)
tmp['x'] = tmp['x'] // 1
tmp['y'] = tmp['y'] // 1
tmp['z'] = tmp['z'] // 1
tmp = tmp[['x', 'y', 'z', 'kinEn']]
tmp

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming your DataFrame is named 'tmp'
# Step 1: Sum kinEn over each x, y pair
result = tmp.groupby(['y', 'z'])['kinEn'].sum().reset_index()

# Step 2: Pivot the data to create a 2D grid for the heatmap
heatmap_data = result.pivot(index='z', columns='y', values='kinEn')

# Step 3: Create the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data, cmap='viridis', annot=False, cbar_kws={'label': 'Sum of kinEn'})
plt.title('Heatmap of Summed kinEn over x and y')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming your DataFrame is named 'tmp'
# Step 1: Sum kinEn over each x, y pair
result = tmp.groupby(['x', 'z'])['kinEn'].sum().reset_index()

# Step 2: Pivot the data to create a 2D grid for the heatmap
heatmap_data = result.pivot(index='z', columns='x', values='kinEn')

# Step 3: Create the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data, cmap='viridis', annot=False, cbar_kws={'label': 'Sum of kinEn'})
plt.title('Heatmap of Summed kinEn over x and y')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming your DataFrame is named 'tmp'
# Step 1: Sum kinEn over each x, y pair
result = tmp.groupby(['x', 'y'])['kinEn'].sum().reset_index()

# Step 2: Pivot the data to create a 2D grid for the heatmap
heatmap_data = result.pivot(index='y', columns='x', values='kinEn')

# Step 3: Create the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data, cmap='viridis', annot=False, cbar_kws={'label': 'Sum of kinEn'})
plt.title('Heatmap of Summed kinEn over x and y')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(projection='3d')
ax.scatter(kev250['x'], kev250['y'], kev250['z'])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title("250 кэВ")
ax.view_init(elev=0, azim=45, roll=0)
plt.tight_layout()
plt.show()

300 кэВ

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(projection='3d')
ax.scatter(kev300['x'], kev300['y'], kev300['z'])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title("300 кэВ")
ax.view_init(elev=0, azim=45, roll=0)
plt.tight_layout()
plt.show()

## Определение вида пятна генерируемого излучения

### Энергия частиц 300 кэВ

Детектор на расстоянии 1м

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.covariance import MinCovDet
from matplotlib.patches import Ellipse

def detect_elliptical_boundary(kev300_2, granularity=50, shape_tolerance=2.0, title=None):
    # Data preparation
    kev300_2['x_trunc'] = (kev300_2['x'] // granularity).round() * granularity
    kev300_2['y_trunc'] = (kev300_2['y'] // granularity).round() * granularity
    heatmap = kev300_2.pivot_table(index='y_trunc', columns='x_trunc', 
                                 values='kinEn', aggfunc='sum').fillna(0)
    
    # Calculate top4 mean threshold
    flat_values = heatmap.values.flatten()
    flat_values = flat_values[~np.isnan(flat_values)]
    top4_values = np.sort(flat_values)[-4:]
    top4_mean = np.mean(top4_values)
    threshold = top4_mean / np.e  # 1/e of top4 mean
    
    # Create initial mask
    mask = heatmap.values >= threshold
    
    # Robust ellipse fitting using minimum covariance determinant
    points = np.argwhere(mask)
    if len(points) < 10:
        raise ValueError("Not enough points for ellipse detection")
    
    robust_cov = MinCovDet().fit(points)
    center = robust_cov.location_
    cov_matrix = robust_cov.covariance_
    
    # Calculate ellipse parameters
    vals, vecs = np.linalg.eigh(cov_matrix)
    order = vals.argsort()[::-1]
    vals, vecs = vals[order], vecs[:, order]
    theta = np.degrees(np.arctan2(*vecs[:, 0]))
    width, height = 2 * shape_tolerance * np.sqrt(vals)
    
    # Calculate and print semi-axes (half-width and half-height)
    semi_width = width / 2 * granularity
    semi_height = height / 2 * granularity
    print(f'Полуширина эллипса: {semi_width:.2f} мм')
    print(f'Полудлина эллипса: {semi_height:.2f} мм')
    
    # Create visualization
    plt.figure(figsize=(16, 9))
    plt.title(f'{title}')
    ax = sns.heatmap(heatmap, cmap='icefire')
    
    # Plot boundary points first
    boundary_y, boundary_x = np.where(mask)
    ax.scatter(boundary_x, boundary_y, s=15, 
              color='white', edgecolor='black', 
              linewidth=0.3, alpha=0.6, zorder=1,
              label=f'1/e Threshold')
    
    # Draw fitted ellipse on top
    ell = Ellipse(xy=(center[1], center[0]),
                  width=width, height=height,
                  angle=theta, 
                  fill=False, color='red', 
                  linewidth=2.5, zorder=2,
                  label='Ellipse Fit')

    ax.add_patch(ell)
    
    plt.ylabel('Y (mm)')
    plt.xlabel('X (mm)')
    plt.legend()
    plt.tight_layout()
    return plt

plot = detect_elliptical_boundary(kev300_2, 
                                  shape_tolerance=2, 
                                  title='Тепловая карта суммарной энергии в детекторе (1 м от мишени) - 300 кэВ')
plot.show()

In [ ]:
query = (np.abs(kev300_2['x']) < 800) & (np.abs(kev300_2['y']) < 800)
plot = detect_elliptical_boundary(kev300_2[query], 
                                  shape_tolerance=2, 
                                  title = 'Тепловая карта суммарной энергии в детекторе (1 м от мишени) - 300 кэВ')
plot.show()

Детектор на расстоянии 0.7м

In [ ]:
plot = detect_elliptical_boundary(kev300_1, 
                                  shape_tolerance=2, 
                                  title = 'Тепловая карта суммарной энергии в детекторе (0.7 м от мишени) - 300 кэВ')
plot.show()

In [ ]:
query = (np.abs(kev300_1['x']) < 800) & (np.abs(kev300_1['y']) < 800)
plot = detect_elliptical_boundary(kev300_1[query], 
                                  shape_tolerance=2, 
                                  title = 'Тепловая карта суммарной энергии в детекторе (0.7 м от мишени) - 300 кэВ')
plot.show()

## 250 кэВ

Детектор на расстоянии 1м

In [ ]:
plot = detect_elliptical_boundary(kev250_2, 
                                  shape_tolerance=2, 
                                  title = 'Тепловая карта суммарной энергии в детекторе (1 м от мишени) - 250 кэВ')
plot.show()

In [ ]:
query = (np.abs(kev250_2['x']) < 800) & (np.abs(kev250_2['y']) < 800)
plot = detect_elliptical_boundary(kev250_2[query], 
                                  shape_tolerance=2, 
                                  title = 'Тепловая карта суммарной энергии в детекторе (1 м от мишени) - 250 кэВ')
plot.show()

In [ ]:
plot = detect_elliptical_boundary(kev250_1, 
                                  shape_tolerance=2, 
                                  title = 'Тепловая карта суммарной энергии в детекторе (0.7 м от мишени) - 250 кэВ')
plot.show()

In [ ]:
query = (np.abs(kev250_1['x']) < 800) & (np.abs(kev250_1['y']) < 800)
plot = detect_elliptical_boundary(kev250_1[query], 
                                  shape_tolerance=2, 
                                  title = 'Тепловая карта суммарной энергии в детекторе (0.7 м от мишени) - 250 кэВ')
plot.show()

Разрежем сферу параллелепипедом вдось x = 15, x = -15, тем самым выделив частицы для исследования углового распределения по энергиям

In [ ]:
kev300 = kev300_0[(kev300_0['x'] > -15) & (kev300_0['x'] < 15)].drop(columns='volumeName')
kev250 = kev250_0[(kev250_0['x'] > -15) & (kev250_0['x'] < 15)].drop(columns='volumeName')

Распределение энергии излучаемых частиц

In [ ]:
fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot()
sns.histplot(data = kev300, x = 'kinEn', bins = 32, stat = 'density')
ax.set_title("300 кэВ Гистограмма энергий гамма-квантов - детектор на выходе из мишени")
ax.set_xlabel('Энергия, кэВ')
ax.set_ylabel('Плотность')
plt.grid()
plt.show()

In [ ]:
fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot()
sns.histplot(data = kev250, x = 'kinEn', bins = 32, stat = 'density')
ax.set_title("250 кэВ Гистограмма энергий гамма-квантов - детектор на выходе из мишени")
ax.set_xlabel('Энергия, кэВ')
ax.set_ylabel('Плотность')
plt.grid()
plt.show()

Построим относительную диаграмму направленности

In [ ]:
theta = np.arctan(kev300['y'] / kev300['z']) * 180 / np.pi
fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot()
sns.histplot(x = theta, bins = 32, stat = 'density')
ax.set_title("300 кэВ Распределение частиц по углам - детектор на выходе из мишени")
ax.set_xlabel('Угол, гр.')
ax.set_ylabel('Плотность')
plt.grid()
plt.show()

In [ ]:
theta = np.arctan(kev250['y'] / kev250['z']) * 180 / np.pi
fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot()
sns.histplot(x = theta, bins = 32, stat = 'density')
ax.set_title("250 кэВ Распределение частиц по углам - детектор на выходе из мишени")
ax.set_xlabel('Угол, гр.')
ax.set_ylabel('Плотность')
plt.grid()
plt.show()

In [ ]:
kev300_ang = kev300.copy()
kev300_ang['theta'] = np.round(np.arctan(kev300_ang['y'] / kev300_ang['z']) * 180 / np.pi, 0)
kev300_ang = kev300_ang.drop('x', axis=1)
kev300_ang = kev300_ang.drop('y', axis=1)
kev300_ang = kev300_ang.drop('z', axis=1)
kev300_ang = kev300_ang.drop('trackNum', axis=1)
kev300_ang = kev300_ang.drop('particleName', axis=1)
kev300_ang = kev300_ang.drop('IsFirstStepInVolume', axis=1)
kev300_ang = kev300_ang.groupby(by = 'theta').mean()
kev300_ang = kev300_ang.reset_index()

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(projection='polar')
ax.plot(kev300_ang['theta'] * np.pi / 180, kev300_ang['kinEn'])
ax.set_xlabel('Зависимость средней энергии (кэВ) от угла (гр.)')
ax.set_title("300 кэВ")
ax.set_theta_zero_location('W', offset=-90)
plt.tight_layout()
plt.show()

In [ ]:
kev250_ang = kev250.copy()
kev250_ang['theta'] = np.round(np.arctan(kev250_ang['y'] / kev250_ang['z']) * 180 / np.pi,  0)
kev250_ang = kev250_ang.drop('x', axis=1)
kev250_ang = kev250_ang.drop('y', axis=1)
kev250_ang = kev250_ang.drop('z', axis=1)
kev250_ang = kev250_ang.drop('trackNum', axis=1)
kev250_ang = kev250_ang.drop('particleName', axis=1)
kev250_ang = kev250_ang.drop('IsFirstStepInVolume', axis=1)
kev250_ang = kev250_ang.groupby(by = 'theta').mean()
kev250_ang = kev250_ang.reset_index()

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(projection='polar')
ax.plot(kev250_ang['theta'] * np.pi / 180, kev250_ang['kinEn'])
ax.set_xlabel('Зависимость средней энергии (кэВ) от угла (гр.)')
ax.set_title("250 кэВ")
ax.set_theta_zero_location('W', offset=-90)
plt.show()

In [ ]:
kev300_ang = kev300.copy()
kev300_ang['theta'] = np.round(np.arctan(kev300_ang['y'] / kev300_ang['z']) * 180 / np.pi / 4) * 4
kev300_ang = kev300_ang.drop('x', axis=1)
kev300_ang = kev300_ang.drop('y', axis=1)
kev300_ang = kev300_ang.drop('z', axis=1)
kev300_ang = kev300_ang.drop('trackNum', axis=1)
kev300_ang = kev300_ang.drop('particleName', axis=1)
kev300_ang = kev300_ang.drop('IsFirstStepInVolume', axis=1)
kev300_ang = kev300_ang.groupby(by = 'theta').mean()
kev300_ang = kev300_ang.reset_index()

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(projection='polar')
ax.plot(kev300_ang['theta'] * np.pi / 180, kev300_ang['kinEn'])
ax.set_xlabel('Зависимость средней энергии (кэВ) от угла (гр.)')
ax.set_title("300 кэВ")
ax.set_theta_zero_location('W', offset=-90)
plt.show()

In [ ]:
kev250_ang = kev250.copy()
kev250_ang['theta'] = np.round(np.arctan(kev250_ang['y'] / kev250_ang['z']) * 180 / np.pi / 4) * 4
kev250_ang = kev250_ang.drop('x', axis=1)
kev250_ang = kev250_ang.drop('y', axis=1)
kev250_ang = kev250_ang.drop('z', axis=1)
kev250_ang = kev250_ang.drop('trackNum', axis=1)
kev250_ang = kev250_ang.drop('particleName', axis=1)
kev250_ang = kev250_ang.drop('IsFirstStepInVolume', axis=1)
kev250_ang = kev250_ang.groupby(by = 'theta').mean()
kev250_ang = kev250_ang.reset_index()

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(projection='polar')
ax.plot(kev250_ang['theta'] * np.pi / 180, kev250_ang['kinEn'])
ax.set_xlabel('Зависимость средней энергии (кэВ) от угла (гр.)')
ax.set_title("250 кэВ")
ax.set_theta_zero_location('W', offset=-90)
plt.show()